# Agentic E-commerce Journey

This notebook demonstrates a complete 3-step agentic e-commerce journey using MCP (Model Context Protocol) tools orchestrated through the Open Responses API.

## Journey Overview

1. **Product Search**: Search for products using Shopify's catalog search MCP tool
2. **Scene Generation**: Create contextual images showing how the product would look in a real environment
3. **Payment Confirmation**: Fetch and display payment and order details using Razorpay MCP tool

In [9]:
# Required imports
from openai import OpenAI
import json
import time
from typing import Iterator, Dict, Any

# API Key Configuration for Different Models
# Update these with your actual API keys

# Step 1: LLAMA model API key (TogetherAI)
LLAMA_API_KEY = "YOUR_TOGETHERAI_API_KEY"

# Step 2: OpenAI model API key 
OPENAI_API_KEY = "YOUR_OPENAI_API_KEY"

# Step 3: Claude model API key (Anthropic)
CLAUDE_API_KEY = "YOUR_ANTHROPIC_API_KEY"

# Additional API keys
RAZORPAY_API_KEY = "YOUR_RAZORPAY_API_KEY"

# Base URL for Open Responses API
OPEN_RESPONSES_BASE_URL = "http://open-responses:6644/v1"

# Create OpenAI clients for each model with their respective API keys
def create_client(api_key: str) -> OpenAI:
    """Create OpenAI client with custom base URL and API key"""
    return OpenAI(
        api_key=api_key,
        base_url=OPEN_RESPONSES_BASE_URL
    )

# Initialize clients for each model
llama_client = create_client(LLAMA_API_KEY)
openai_client = create_client(OPENAI_API_KEY)
claude_client = create_client(CLAUDE_API_KEY)

print("Setup complete! Ready to start the agentic e-commerce journey.")
print("🔧 OpenAI clients initialized for all models.")
print("⚠️ Remember to update all API keys before running the journey!")

Setup complete! Ready to start the agentic e-commerce journey.
🔧 OpenAI clients initialized for all models.
⚠️ Remember to update all API keys before running the journey!


In [12]:
# Custom Streaming Event Handler for Open Responses API
def handle_streaming_events(stream, step_name: str = "Process"):
    """
    Handle streaming events from Open Responses API
    Properly parses event objects and extracts content
    Returns the accumulated response text
    """
    full_response = ""
    response_started = False
    
    print(f"🔄 Starting {step_name}...")
    
    try:
        for event in stream:
            # Extract event type from the event object
            event_type = getattr(event, 'type', None)
            
            if event_type:
                # Handle specific event types based on the new format
                if event_type == "response.created":
                    print(f"📝 Response created")
                    response_started = True
                    
                elif event_type == "response.in_progress":
                    print(f"⚙️  In progress...")
                    
                elif "executing" in event_type or "in_progress" in event_type:
                    # Extract tool name if available
                    tool_name = event_type
                    print(f"⚙️  Executing {tool_name}...")
                    
                elif "completed" in event_type:
                    if event_type == "response.completed":
                        print(f"✅ Response completed")
                    else:
                        tool_name = event_type
                        print(f"✅ {tool_name} completed")
                    
                elif event_type == "response.output_text.delta":
                    # Extract delta content from the event
                    if hasattr(event, 'delta'):
                        delta_content = event.delta
                        print(delta_content, end='', flush=True)
                        full_response += delta_content
                    # Also check if it's in a nested structure
                    elif hasattr(event, 'choices') and event.choices:
                        if hasattr(event.choices[0], 'delta') and hasattr(event.choices[0].delta, 'content'):
                            content = event.choices[0].delta.content
                            if content:
                                print(content, end='', flush=True)
                                full_response += content
                                
                elif event_type == "response.output_text.done":
                    # Final text output - we've already streamed it
                    pass
                    
                elif event_type == "response.output_item.added":
                    print(f"🔧 Tool call added")
                    
                # For other events, show minimal info
                elif not any(x in event_type for x in ["sequence_number", "isValid"]):
                    print(f"🔍 {event_type}")
            else:
                # Try to extract content from choices/delta structure (fallback)
                if hasattr(event, 'choices') and event.choices:
                    if hasattr(event.choices[0], 'delta') and hasattr(event.choices[0].delta, 'content'):
                        content = event.choices[0].delta.content
                        if content:
                            print(content, end='', flush=True)
                            full_response += content
                    
    except Exception as e:
        print(f"\n⚠️ Error processing stream: {e}")
        import traceback
        traceback.print_exc()
        
    if not response_started:
        print("⚠️ No response.created event detected")
        
    print(f"\n\n✅ {step_name} completed!")
    return full_response

print("🔧 Custom streaming event handler loaded!")

🔧 Custom streaming event handler loaded!


## Step 1: Product Search

In this step, we use the Shopify MCP tool to search for products in the catalog. The `search_shop_catalog` tool will help us find products and retrieve complete details including images.

### Configuration:
- **Model**: `togetherai@meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8`
- **MCP Server**: Shopify at `https://allbirds.com/api/mcp`
- **Tool**: `search_shop_catalog`

In [13]:
# Step 1: Search for HP printer using Shopify MCP tool (LLAMA Model)
def search_product():
    print("🔍 Searching for HP printer using LLAMA model...")
    print(f"🔑 Using API Key: {LLAMA_API_KEY[:10]}..." if LLAMA_API_KEY != "YOUR_TOGETHERAI_API_KEY" else "⚠️ Please set LLAMA_API_KEY")
    
    try:
        # Create streaming response using OpenAI SDK
        stream = llama_client.responses.create(
            model="togetherai@meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8",
            tools=[
                {
                    "type": "mcp",
                    "server_label": "shopify",
                    "server_url": "https://allbirds.com/api/mcp",
                    "allowed_tools": ["search_shop_catalog"]
                }
            ],
            instructions=(
                "Use search_shop_catalog to search the product and then bring complete details "
                "of the product including all available images. Do not mention names of tools in the response"
            ),
            input="Find me one cheapest men's sneaker",
            stream=True
        )
        
        # Use custom streaming handler to process events
        full_response = handle_streaming_events(stream, "Product Search")
        return full_response
        
    except Exception as e:
        print(f"\n❌ Error during product search: {e}")
        return None

# Execute the product search (uncomment to run)
search_result = search_product()

🔍 Searching for HP printer using LLAMA model...
🔑 Using API Key: tgp_v1_JNQ...
🔄 Starting Product Search...
📝 Response created
⚙️  In progress...
🔧 Tool call added
⚙️  Executing response.mcp_call.shopify_search_shop_catalog.in_progress...
⚙️  Executing response.mcp_call.shopify_search_shop_catalog.executing...
✅ response.mcp_call.shopify_search_shop_catalog.completed completed
The cheapest men's sneaker is the Men's Tree Runner - Mist (White Sole) priced at $100. It is a breathable and lightweight sneaker made with responsibly sourced eucalyptus tree fiber. The shoe is available in various sizes (8-14) and has multiple images available.

Here are the details of the product:

* Title: Men's Tree Runner - Mist (White Sole)
* Price: $100
* Description: The Allbirds Tree Runner is a breathable and lightweight sneaker made with responsibly sourced eucalyptus tree fiber that feels silky smooth and cool on your skin. These shoes are perfect for everyday casual wear, walking, and warmer weathe

## Step 2: Scene Generation for Product Selection

In this step, we create a contextual image showing how the selected product would look in a real environment (study table). This uses image generation MCP tools to help customers visualize the product in their space.

### Configuration:
- **Model**: `openai@gpt-4.1-mini`
- **MCP Server**: Image generation at if running without docker `http://localhost:8086/mcp` if running within docker `http://demo-mcp-server:8086/mcp`
- **Tools**: `image_to_base64`, `img_scene_generator`

In [15]:
# Step 2: Generate scene image showing printer on study table (OpenAI Model)
def generate_scene():
    print("🎨 Generating scene image using OpenAI model...")
    print(f"🔑 Using API Key: {OPENAI_API_KEY[:10]}..." if OPENAI_API_KEY != "YOUR_OPENAI_API_KEY" else "⚠️ Please set OPENAI_API_KEY")
    
    try:
        # Create streaming response using OpenAI SDK with structured output
        stream = openai_client.responses.create(
            model="openai@gpt-4.1-mini",
            tools=[
                {
                    "type": "mcp",
                    "server_label": "img_mcp",
                    "server_url": "http://demo-mcp-server:8086/mcp",
                    "allowed_tools": [
                        "image_to_base64",
                        "img_scene_generator"
                    ]
                }
            ],
            instructions=(
                "Use tool image_to_base64 to generate base64encoded String from provided URL of image "
                "and then use tool img_scene_generator to generate the image for the scene with input prompt "
                "and encodedFilePath returned by the tool image_to_base64. If the image is generated then return the generated scene image url."
            ),
            input=(
                "How it will look like on my study table. Create new image of described situation using tools "
                "and object available here https://cdn.shopify.com/s/files/1/1104/4168/files/TR3MMST080_SHOE_LEFT_GLOBAL_MENS_TREE_RUNNER_MIST_WHITE_9959c126-77c8-42fc-aef3-7d6093c605bf.png?v=1751166590"
            ),
            stream=True
        )
        
        # Use custom streaming handler to process events
        full_response = handle_streaming_events(stream, "Scene Generation")
        
        # # Try to parse the JSON response
        # try:
        #     scene_data = json.loads(full_response.strip())
        #     print(f"\n📋 Scene Description: {scene_data.get('situationDescription', 'N/A')}")
        #     print(f"🖼️ Generated Image URL: {scene_data.get('image_url', 'N/A')}")
        #     print(f"✅ Completed Stage: {scene_data.get('completedStage', 'N/A')}")
        #     return scene_data
        # except json.JSONDecodeError:
        #     print("⚠️ Could not parse JSON response")
        #     return {"raw_response": full_response}
            
    except Exception as e:
        print(f"\n❌ Error during scene generation: {e}")
        return None

# Execute the scene generation (uncomment to run)
scene_result = generate_scene()

🎨 Generating scene image using OpenAI model...
🔑 Using API Key: sk-proj-DZ...
🔄 Starting Scene Generation...
📝 Response created
⚙️  In progress...
🔧 Tool call added
⚙️  Executing response.mcp_call.img_mcp_image_to_base64.in_progress...
⚙️  Executing response.mcp_call.img_mcp_image_to_base64.executing...
✅ response.mcp_call.img_mcp_image_to_base64.completed completed
🔧 Tool call added
⚙️  Executing response.mcp_call.img_mcp_img_scene_generator.in_progress...
⚙️  Executing response.mcp_call.img_mcp_img_scene_generator.executing...
✅ response.mcp_call.img_mcp_img_scene_generator.completed completed
Here is the generated image of how the Men's Tree Runner Mist White shoes will look on your study table with typical study items like books, a laptop, and a cup of coffee, all under warm and inviting lighting: 
http://localhost:8086/generated_image_1754632396.png Response completed


✅ Scene Generation completed!


## Step 3: Payment Confirmation

In this final step, we fetch payment and order details using Razorpay MCP tools. This demonstrates how to retrieve transaction information and present it in a user-friendly format.

### Configuration:
- **Model**: `claude@claude-sonnet-4-20250514`
- **MCP Server**: Razorpay at `https://mcp.razorpay.com/sse`
- **Tools**: `fetch_payment`, `fetch_order`
- **Authentication**: Bearer token required in headers

In [8]:
# Step 3: Fetch payment and order details using Razorpay MCP tools (Claude Model)
def confirm_payment():
    print("💳 Fetching payment and order details using Claude model...")
    print(f"🔑 Using Claude API Key: {CLAUDE_API_KEY[:10]}..." if CLAUDE_API_KEY != "YOUR_ANTHROPIC_API_KEY" else "⚠️ Please set CLAUDE_API_KEY")
    print(f"🔑 Using Razorpay API Key: {RAZORPAY_API_KEY[:10]}..." if RAZORPAY_API_KEY != "YOUR_RAZORPAY_API_KEY" else "⚠️ Please set RAZORPAY_API_KEY")
    
    try:
        # Create streaming response using OpenAI SDK
        stream = claude_client.responses.create(
            model="claude@claude-sonnet-4-20250514",
            tools=[
                {
                    "type": "mcp",
                    "server_label": "razorpay",
                    "server_url": "https://mcp.razorpay.com/sse",
                    "allowed_tools": [
                        "fetch_payment",
                        "fetch_order"
                    ],
                    "headers": {
                        "Authorization": f"Bearer {RAZORPAY_API_KEY}"
                    }
                }
            ],
            instructions=(
                "Use fetch_payment tool by passing payment id provided by the user, "
                "get the order id from payment details response then fetch order details "
                "using fetch_order tool and return order details in markdown format"
            ),
            input="payment id {payment_id}",
            store=True,
            stream=True
        )
        
        # Use custom streaming handler to process events
        full_response = handle_streaming_events(stream, "Payment Confirmation")
        return full_response
        
    except Exception as e:
        print(f"\n❌ Error during payment confirmation: {e}")
        return None

# Execute the payment confirmation (uncomment to run)
payment_result = confirm_payment()

💳 Fetching payment and order details using Claude model...
🔑 Using Claude API Key: sk-ant-api...
🔑 Using Razorpay API Key: cnpwX2xpdm...
🔄 Starting Payment Confirmation...
📝 Response created
⚙️  In progress...
I see you've provided a placeholder `{payment_id}` rather than an actual payment ID. To fetch the payment and order details, I need the actual payment ID value.

Could you please provide the specific payment ID you'd like me to look up? It should be a string of characters that uniquely identifies the payment in Razorpay's system.✅ Response completed


✅ Payment Confirmation completed!


## Quick Test Functions

Uncomment any of the lines below to test individual functions:

```python
# Test individual functions:
search_result = search_product()
scene_result = generate_scene()
payment_result = confirm_payment()
```

### Expected Outputs:
- **Step 1**: Product details with images (powered by LLAMA)
- **Step 2**: JSON response with scene description and generated image URL (powered by OpenAI)
- **Step 3**: Payment and order details in markdown format (powered by Claude)